In [1]:
import pandas as pd
import featuretools as ft
from woodwork.logical_types import Categorical

In [2]:
# let's load the data again

df = pd.read_csv("retail.csv", parse_dates=["invoice_date"])

df.head()

,customer_id,invoice,invoice_date,stock_code,description,quantity,price
0,13085.0,489434,2009-12-01 07:45:00,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95
1,13085.0,489434,2009-12-01 07:45:00,79323P,PINK CHERRY LIGHTS,12,6.75
2,13085.0,489434,2009-12-01 07:45:00,79323W,WHITE CHERRY LIGHTS,12,6.75
3,13085.0,489434,2009-12-01 07:45:00,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10
4,13085.0,489434,2009-12-01 07:45:00,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25


In [3]:
# create and entity set

es = ft.EntitySet(id="data")

In [4]:
# Add the data to the entity

es = es.add_dataframe(
    dataframe=df,              # the dataframe with the data
    dataframe_name="data",     # unique name to associate with this dataframe
    index="rows",              # column name to index the items
    make_index=True,           # if true, create a new column with unique values
    time_index="invoice_date", # column containing time data
    logical_types={
        "customer_id": Categorical, # the id is numerical, but should be handled as categorical
        "invoice": Categorical,
    },
)

C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarnin

In [5]:
es["data"].ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
rows,int64,Integer,['index']
customer_id,category,Categorical,['category']
invoice,category,Categorical,['category']
invoice_date,datetime64[ns],Datetime,['time_index']
stock_code,category,Categorical,['category']
description,category,Categorical,['category']
quantity,int64,Integer,['numeric']
price,float64,Double,['numeric']


In [6]:
# Create a new dataframe with invoices
# indicating its relationship to the main data

es.normalize_dataframe(
    base_dataframe_name="data",     # Datarame name from which to split.
    new_dataframe_name="invoices",  # Name of the new dataframe.
    index="invoice",                # relationship will be created across this column.
    copy_columns=["customer_id"],   # columns to remove from base_dataframe and move to new dataframe.
)

Entityset: data
  DataFrames:
    data [Rows: 741301, Columns: 8]
    invoices [Rows: 40505, Columns: 3]
  Relationships:
    data.invoice -> invoices.invoice

In [7]:
# the original data

es["data"].head()

,rows,customer_id,invoice,invoice_date,stock_code,description,quantity,price
0,0,13085.0,489434,2009-12-01 07:45:00,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95
1,1,13085.0,489434,2009-12-01 07:45:00,79323P,PINK CHERRY LIGHTS,12,6.75
2,2,13085.0,489434,2009-12-01 07:45:00,79323W,WHITE CHERRY LIGHTS,12,6.75
3,3,13085.0,489434,2009-12-01 07:45:00,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10
4,4,13085.0,489434,2009-12-01 07:45:00,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25


In [8]:
# cumulative transform primitives

cum_primitives = ["cum_sum", "cum_max", "diff", "time_since_previous"]

general_primitives = ["sine", "cosine"]

In [9]:
# if you want to double check the result of the
# feature creation process, set features_only=True

feature_defs = ft.dfs(
    entityset=es,                                # the entity set
    target_dataframe_name="data",                # the dataframe for wich to create the features
    agg_primitives=[],                           # empty list to avoid returning the defo parameters
    trans_primitives=general_primitives,         # empty list to avoid returning the defo parameters
    groupby_trans_primitives = cum_primitives,   # the operations to perform by invoice
    ignore_dataframes = ["invoices"],            # columns to ignore when creating features
    features_only=True,    
)

# display name of created features
feature_defs

[<Feature: customer_id>,
 <Feature: invoice>,
 <Feature: stock_code>,
 <Feature: description>,
 <Feature: quantity>,
 <Feature: price>,
 <Feature: COSINE(price)>,
 <Feature: COSINE(quantity)>,
 <Feature: SINE(price)>,
 <Feature: SINE(quantity)>,
 <Feature: CUM_MAX(price) by invoice>,
 <Feature: CUM_MAX(quantity) by invoice>,
 <Feature: CUM_SUM(price) by invoice>,
 <Feature: CUM_SUM(quantity) by invoice>,
 <Feature: DIFF(price) by invoice>,
 <Feature: DIFF(quantity) by invoice>,
 <Feature: TIME_SINCE_PREVIOUS(invoice_date) by invoice>]

In [10]:
# create all features simultaneously

feature_matrix, feature_defs = ft.dfs(
    entityset=es,                                # the entity set
    target_dataframe_name="data",                # the dataframe for wich to create the features
    agg_primitives=[],                           # empty list to avoid returning the defo parameters
    trans_primitives=general_primitives,         # empty list to avoid returning the defo parameters
    groupby_trans_primitives = cum_primitives,   # the operations to perform by invoice
    ignore_dataframes = ["invoices"],            # columns to ignore when creating features
)

# display name of created features
feature_defs

C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:545: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(groupby)
C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:588: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform t

[<Feature: customer_id>,
 <Feature: invoice>,
 <Feature: stock_code>,
 <Feature: description>,
 <Feature: quantity>,
 <Feature: price>,
 <Feature: COSINE(price)>,
 <Feature: COSINE(quantity)>,
 <Feature: SINE(price)>,
 <Feature: SINE(quantity)>,
 <Feature: CUM_MAX(price) by invoice>,
 <Feature: CUM_MAX(quantity) by invoice>,
 <Feature: CUM_SUM(price) by invoice>,
 <Feature: CUM_SUM(quantity) by invoice>,
 <Feature: DIFF(price) by invoice>,
 <Feature: DIFF(quantity) by invoice>,
 <Feature: TIME_SINCE_PREVIOUS(invoice_date) by invoice>]

In [11]:
# Resulting datatable with original 
# and new features

feature_matrix.head()

,customer_id,invoice,stock_code,description,quantity,price,COSINE(price),COSINE(quantity),SINE(price),SINE(quantity),CUM_MAX(price) by invoice,CUM_MAX(quantity) by invoice,CUM_SUM(price) by invoice,CUM_SUM(quantity) by invoice,DIFF(price) by invoice,DIFF(quantity) by invoice,TIME_SINCE_PREVIOUS(invoice_date) by invoice
rows,,,,,,,,,,,,,,,,,
0,13085.0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,0.785796,0.843854,0.618486,-0.536573,6.95,12.0,6.95,12.0,NaN,NaN,NaN
1,13085.0,489434,79323P,PINK CHERRY LIGHTS,12,6.75,0.893006,0.843854,0.450044,-0.536573,6.95,12.0,13.70,24.0,-0.20,0.0,0.0
2,13085.0,489434,79323W,WHITE CHERRY LIGHTS,12,6.75,0.893006,0.843854,0.450044,-0.536573,6.95,12.0,20.45,36.0,0.00,0.0,0.0
3,13085.0,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,-0.504846,-0.640144,0.863209,-0.768255,6.95,48.0,22.55,84.0,-4.65,36.0,0.0
4,13085.0,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,0.315322,0.424179,0.948985,-0.905578,6.95,48.0,23.80,108.0,-0.85,-24.0,0.0


In [12]:
feature_matrix.shape

(741301, 17)

## In relation to pandas

In [13]:
import numpy as np
import pandas as pd

In [14]:
# load data

df = pd.read_csv("retail.csv", parse_dates=["invoice_date"])

df.head()

,customer_id,invoice,invoice_date,stock_code,description,quantity,price
0,13085.0,489434,2009-12-01 07:45:00,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95
1,13085.0,489434,2009-12-01 07:45:00,79323P,PINK CHERRY LIGHTS,12,6.75
2,13085.0,489434,2009-12-01 07:45:00,79323W,WHITE CHERRY LIGHTS,12,6.75
3,13085.0,489434,2009-12-01 07:45:00,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10
4,13085.0,489434,2009-12-01 07:45:00,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25


In [15]:
# Cumulative sum at invoice level

# numerical variables
numeric_vars = ["quantity", "price"]

# the cumulative functions
func = ["cumsum", "cummax", "diff"]

# new variable names
new_names = [f"{var}_{function}" for function in func for var in numeric_vars]

new_names

['quantity_cumsum',
 'price_cumsum',
 'quantity_cummax',
 'price_cummax',
 'quantity_diff',
 'price_diff']

In [16]:
df[new_names] = df.groupby(
    "invoice")[numeric_vars].agg(func)

In [17]:
# inspect values for 1 invoice

df[df["invoice"] == "489434" ][numeric_vars + new_names].head()

,quantity,price,quantity_cumsum,price_cumsum,quantity_cummax,price_cummax,quantity_diff,price_diff
0,12,6.95,12,12,NaN,6.95,6.95,NaN
1,12,6.75,24,12,0.0,13.70,6.95,-0.20
2,12,6.75,36,12,0.0,20.45,6.95,0.00
3,48,2.10,84,48,36.0,22.55,6.95,-4.65
4,24,1.25,108,48,-24.0,23.80,6.95,-0.85


In [18]:
# general transformations

new_names = [f"{var}_{function}" for function in ["sin", "cos"]
             for var in numeric_vars]

new_names

['quantity_sin', 'price_sin', 'quantity_cos', 'price_cos']

In [19]:
df[new_names] = df[numeric_vars].agg([np.sin, np.cos])

df[new_names].head()

C:\Users\Sole\AppData\Local\Temp\ipykernel_6776\866113479.py:1: FutureWarning: using <ufunc 'sin'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df[new_names] = df[numeric_vars].agg([np.sin, np.cos])
C:\Users\Sole\AppData\Local\Temp\ipykernel_6776\866113479.py:1: FutureWarning: using <ufunc 'cos'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df[new_names] = df[numeric_vars].agg([np.sin, np.cos])


,quantity_sin,price_sin,quantity_cos,price_cos
0,-0.536573,0.843854,0.618486,0.785796
1,-0.536573,0.843854,0.450044,0.893006
2,-0.536573,0.843854,0.450044,0.893006
3,-0.768255,-0.640144,0.863209,-0.504846
4,-0.905578,0.424179,0.948985,0.315322
